In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import glob

%matplotlib inline
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

# CSV 파일들을 저장할 빈 DataFrame을 생성합니다.


# data 폴더에 있는 모든 CSV 파일을 가져옵니다.
csv_files = glob.glob('./high50data/*.csv')
print(csv_files)

['./high50data\\000270.csv', './high50data\\000660.csv', './high50data\\000810.csv', './high50data\\003550.csv', './high50data\\003670.csv', './high50data\\005380.csv', './high50data\\005490.csv', './high50data\\005930.csv', './high50data\\005935.csv', './high50data\\006400.csv', './high50data\\009150.csv', './high50data\\010130.csv', './high50data\\012330.csv', './high50data\\015760.csv', './high50data\\017670.csv', './high50data\\018260.csv', './high50data\\024110.csv', './high50data\\028260.csv', './high50data\\032830.csv', './high50data\\033780.csv', './high50data\\034020.csv', './high50data\\034730.csv', './high50data\\035420.csv', './high50data\\035720.csv', './high50data\\047050.csv', './high50data\\051910.csv', './high50data\\055550.csv', './high50data\\066570.csv', './high50data\\068270.csv', './high50data\\086790.csv', './high50data\\096770.csv', './high50data\\105560.csv', './high50data\\138040.csv', './high50data\\207940.csv', './high50data\\316140.csv', './high50data\\3234

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from itertools import combinations
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import Huber
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.models import Model

stock_num_li = ['005930', '000660', '207940', '005935', '005380', '005490', '000270', '051910', '035420', '006400', '068270', '105560', '012330', '003670', '028260', '055550', '066570', '035720', '032830', '096770', '003550', '086790', '000810','033780', '017670', '015760', '138040', '034730', '009150', '018260', '329180', '010130', '047050', '323410', '316140', '024110', '034020', '352820']
#여기는 다시 돌려야함 '017670', '015760', '138040', '034730', '009150', '018260', '329180', '010130', '047050', '323410', '316140', '024110', '034020', '352820'
normal = 'Open High Low Close Volume '
KSPU = 'Kospi S&P500 bb_upper'
M1 = 'ma10'
DU = 'high_low_diff bb_upper'
M1SP = 'ma10 S&P500'
M2VM = 'ma20 volumn_ma5'
SPU = 'S&P500 bb_upper'
M1K = 'ma10 %K'
M2SP = 'ma20 S&P500'
KOU = 'Kospi bb_upper'

WINDOW_SIZE = 50
BATCH_SIZE = WINDOW_SIZE * 2
epoch_size = 30
patience = 8
learning_rate = 0.003

colset_list = [KSPU, M1, DU, M1SP, SPU, M1K, M2SP, KOU]
result_pridict = []
scaler = MinMaxScaler()
for stock_num in stock_num_li:
  stock = pd.DataFrame()
  file_path  = [csv for csv in csv_files if f'./high50data\\{stock_num}.csv' in csv][0]
  df = pd.read_csv(file_path)
  stock = pd.concat([stock, df], ignore_index=True)

  for colset in colset_list:
    tmp_use_col = normal+colset
    colset = colset.replace(" ", "_")
    use_cols = tmp_use_col.split()
    ln_use_cols =len(use_cols)
    # print(ln_use_cols, use_cols)
    scaled = scaler.fit_transform(stock[use_cols])
    df = pd.DataFrame(scaled, columns=use_cols)


    X, y = [], []

    for i in range(WINDOW_SIZE, len(scaled)):
        X.append(scaled[i - WINDOW_SIZE:i])
        y.append([scaled[i][3]]) # Close 열만 학습
    X, y = np.array(X), np.array(y)

    split_ratio = 0.8
    split_index = int(split_ratio * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
# # '373220'
 # LSTM 모델 정의
    lstm_input = Input(shape=(WINDOW_SIZE, ln_use_cols), name='lstm_input')
    lstm_layer = LSTM(32, activation='tanh')(lstm_input)

    # GRU 모델 정의
    gru_input = Input(shape=(WINDOW_SIZE, ln_use_cols), name='gru_input')
    gru_layer = GRU(32, activation='tanh')(gru_input)

    # 두 모델을 결합
    concatenated = concatenate([lstm_layer, gru_layer])

    # 공통 레이어 추가
    dense_layer = Dense(16, activation='relu')(concatenated)

    # 출력 레이어
    output = Dense(ln_use_cols, name='output')(dense_layer)

    # 모델 생성
    model = Model(inputs=[lstm_input, gru_input], outputs=output)

    loss = Huber()
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=loss, optimizer=optimizer, metrics=[MeanAbsoluteError()])

    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    tmp = '_'.join(use_cols[5:])
    checkpoint = ModelCheckpoint(
        filepath=f'./checkpoint/En/{WINDOW_SIZE}/{stock_num}_{tmp}_{WINDOW_SIZE}_{learning_rate}.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
    )

    history = model.fit(
        {'lstm_input': X_train, 'gru_input': X_train},  # 입력을 딕셔너리로 지정
        y_train,
        validation_split=0.2,
        epochs=epoch_size,
        callbacks=[early_stopping, checkpoint],
        batch_size=BATCH_SIZE
    )

    input_scaled = stock[use_cols]
    total_data = [[input_scaled[len(input_scaled)-WINDOW_SIZE-1:len(input_scaled)-1]]]
    conform_total_data = [[input_scaled[len(input_scaled)-1:len(input_scaled)]]][:4]

    data = np.array(total_data)
    data = data.tolist()
    data = data[0]

    conform_data = conform_total_data[0][0]
    conform_data = conform_data.values[0]

    data_2d = np.array(data).reshape(-1, len(data[0][0]))

    scaler = MinMaxScaler()
    scaler.fit(data_2d)

    scaled_data_2d = scaler.transform(data_2d)
    model = load_model(f'./checkpoint/En/{WINDOW_SIZE}/{stock_num}_{tmp}_{WINDOW_SIZE}_{learning_rate}.h5', custom_objects={'MeanAbsoluteError': MeanAbsoluteError()})
    scaled_data_3d = scaled_data_2d.reshape(len(data), len(data[0]), len(data[0][0]))
    predicted_value = model.predict({'lstm_input': scaled_data_3d, 'gru_input': scaled_data_3d})

    predicted_value_original = scaler.inverse_transform(predicted_value)
    predicted_value_original = np.round(predicted_value_original, ln_use_cols)[0][:4]
    conform_total_data = conform_total_data[0][0].values[0]
    predicted_value_original = list(predicted_value_original)
    predicted_value_original.extend(conform_total_data[:4])
    
    use_cols_check = '/'.join(use_cols)
    predicted_value_original.append(use_cols_check)
    result_use_cols = use_cols[:4]
    result_use_cols.extend(['_Open', '_High', '_Low', '_Close', 'Use_Cols_Check'])

    result_dict = dict(zip(result_use_cols, predicted_value_original))
    result_pridict.append(result_dict)



Epoch 1/30
32/34 [===========================>..] - ETA: 0s - loss: 0.0018 - mean_absolute_error: 0.0388
Epoch 1: val_loss improved from inf to 0.00061, saving model to ./checkpoint/En/50\005930_Kospi_S&P500_bb_upper_50_0.003.h5
34/34 [==============================] - 4s 46ms/step - loss: 0.0017 - mean_absolute_error: 0.0373 - val_loss: 6.1043e-04 - val_mean_absolute_error: 0.0266
Epoch 2/30
33/34 [============================>.] - ETA: 0s - loss: 5.8672e-05 - mean_absolute_error: 0.0084
Epoch 2: val_loss improved from 0.00061 to 0.00014, saving model to ./checkpoint/En/50\005930_Kospi_S&P500_bb_upper_50_0.003.h5
34/34 [==============================] - 1s 24ms/step - loss: 5.7968e-05 - mean_absolute_error: 0.0083 - val_loss: 1.3698e-04 - val_mean_absolute_error: 0.0130
Epoch 3/30
32/34 [===========================>..] - ETA: 0s - loss: 2.2839e-05 - mean_absolute_error: 0.0052
Epoch 3: val_loss improved from 0.00014 to 0.00011, saving model to ./checkpoint/En/50\005930_Kospi_S&P500_bb

In [3]:
print(result_pridict)
import pandas as pd

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(result_pridict)

# DataFrame을 엑셀 파일로 저장
df.to_excel('En_result_pridict_50_0.003_w50.xlsx', index=False)

[{'Open': 79318.37, 'High': 79337.14, 'Low': 77825.11, 'Close': 78667.22, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/Kospi/S&P500/bb_upper'}, {'Open': 79796.45, 'High': 79166.61, 'Low': 78996.1, 'Close': 78332.07, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10'}, {'Open': 78263.414, 'High': 78201.94, 'Low': 78197.25, 'Close': 78916.484, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/high_low_diff/bb_upper'}, {'Open': 78660.94, 'High': 79221.6, 'Low': 78668.695, 'Close': 78487.67, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10/S&P500'}, {'Open': 79147.484, 'High': 79250.31, 'Low': 79019.11, 'Close': 78935.59, '_Open': 78900.0, '_High': 79500.0, '_Low': 78100.0, '_Close': 78300.0, 'Use_Cols_Chec